In [18]:
from datetime import datetime

class Usuario:
    def __init__(self, endereco):
        self.endereco = endereco
        self.contas = []

    def fazer_transacao(self, conta, transacao):
        transacao.registrar(conta)

    def adicionar_conta(self, conta):
        self.contas.append(conta)

class Pessoa(Usuario):
    def __init__(self, nome, nascimento, cpf, endereco):
        super().__init__(endereco)
        self.nome = nome
        self.nascimento = nascimento
        self.cpf = cpf

class ContaBancaria:
    def __init__(self, numero, cliente):
        self._saldo = 0
        self._numero = numero
        self._agencia = "0001"
        self._cliente = cliente
        self._historico = HistoricoTransacoes()

    @classmethod
    def nova_conta(cls, cliente, numero):
        return cls(numero, cliente)

    def get_saldo(self):
        return self._saldo

    def get_numero(self):
        return self._numero

    def get_agencia(self):
        return self._agencia

    def get_cliente(self):
        return self._cliente

    def get_historico(self):
        return self._historico

    def sacar(self, valor):
        saldo = self.get_saldo()
        excedeu_saldo = valor > saldo

        if excedeu_saldo:
            print("\n@@@ Falhou! Sem saldo suficiente. @@@")

        elif valor > 0:
            self._saldo -= valor
            print("\n=== Saque ok! ===")
            return True

        else:
            print("\n@@@ Falhou! Valor inválido. @@@")

        return False

    def depositar(self, valor):
        if valor > 0:
            self._saldo += valor
            print("\n=== Depósito ok! ===")
        else:
            print("\n@@@ Falhou! Valor inválido. @@@")
            return False

        return True

class ContaCorrente(ContaBancaria):
    def __init__(self, numero, cliente, limite=500, limite_saques=3):
        super().__init__(numero, cliente)
        self.limite = limite
        self.limite_saques = limite_saques

    def sacar(self, valor):
        numero_saques = len([transacao for transacao in self.get_historico().transacoes if transacao["tipo"] == "Saque"])

        excedeu_limite = valor > self.limite
        excedeu_saques = numero_saques >= self.limite_saques

        if excedeu_limite:
            print("\n@@@ Falhou! Valor do saque excede o limite. @@@")

        elif excedeu_saques:
            print("\n@@@ Falhou! Limite de saques excedido. @@@")

        else:
            return super().sacar(valor)

        return False

    def __str__(self):
        return f"""\
            Agência:\t{self.get_agencia()}
            Conta:\t\t{self.get_numero()}
            Titular:\t{self.get_cliente().nome}
        """

class HistoricoTransacoes:
    def __init__(self):
        self._transacoes = []

    def get_transacoes(self):
        return self._transacoes

    def adicionar_transacao(self, transacao):
        self._transacoes.append({
            "tipo": transacao.__class__.__name__,
            "valor": transacao.valor,
            "data": datetime.now().strftime("%d-%m-%Y %H:%M:%S"),
        })

class Transacao:
    def valor(self):
        raise NotImplementedError

    def registrar(self, conta):
        raise NotImplementedError

class Saque(Transacao):
    def __init__(self, valor):
        self._valor = valor

    @property
    def valor(self):
        return self._valor

    def registrar(self, conta):
        sucesso = conta.sacar(self.valor)
        if sucesso:
            conta.get_historico().adicionar_transacao(self)

class Deposito(Transacao):
    def __init__(self, valor):
        self._valor = valor

    @property
    def valor(self):
        return self._valor

    def registrar(self, conta):
        sucesso = conta.depositar(self.valor)
        if sucesso:
            conta.get_historico().adicionar_transacao(self)

# Código funcional
usuarios = []
contas_correntes = []
numero_proxima_conta = 1

def adicionar_usuario(nome, nascimento, cpf, endereco):
    for usuario in usuarios:
        if usuario['cpf'] == cpf:
            print(f"Já existe um usuário com o CPF {cpf}.")
            return None
    
    usuario = {
        'nome': nome,
        'nascimento': nascimento,
        'cpf': cpf,
        'endereco': endereco
    }
    usuarios.append(usuario)
    print(f"Usuário {nome} cadastrado com sucesso.")
    return usuario

def adicionar_conta(usuario):
    global numero_proxima_conta
    
    for conta in contas_correntes:
        if conta['usuario']['cpf'] == usuario['cpf']:
            print(f"O usuário {usuario['nome']} já tem uma conta.")
            return None
    
    conta = {
        'agencia': '0001',
        'numero_conta': numero_proxima_conta,
        'usuario': usuario
    }
    contas_correntes.append(conta)
    numero_proxima_conta += 1
    print(f"Conta criada para o usuário {usuario['nome']} - Agência: {conta['agencia']} - Conta: {conta['numero_conta']}.")
    return conta

def mostrar_extrato(saldo, extrato=""):
    if not extrato:
        return "Sem movimentações.", saldo
    else:
        extrato_completo = f"\n================ EXTRATO ================\n{extrato}\nSaldo: R$ {saldo:.2f}\n=========================================="
        return extrato_completo, saldo

def depositar_dinheiro(saldo, valor, extrato):
    if valor <= 0:
        return '\nValor inválido.', saldo, extrato
    else:
        saldo += valor
        extrato += f"Depósito: R$ {valor:.2f}\n"
        return f'\nDepósito ok. Seu saldo é: R$ {saldo:.2f}', saldo, extrato

def sacar_dinheiro(valor, limite, saldo, extrato, limite_saques, numero_saques):
    if numero_saques >= limite_saques:
        return '\nLimite de saques diário atingido.', saldo, extrato, numero_saques
    else:
        if valor <= 0:
            return '\nValor inválido.', saldo, extrato, numero_saques
        elif valor > limite:
            return '\nMáximo para saque é 500.', saldo, extrato, numero_saques
        elif valor > saldo:
            return f'\nSem fundos suficientes: \nValor Saque: R$ {valor:.2f} \nSaldo: R$ {saldo:.2f}', saldo, extrato, numero_saques
        else:
            saldo -= valor
            extrato += f"Saque: R$ {valor:.2f}\n"
            numero_saques += 1
            return f'\nSaque ok. Seu saldo é: R$ {saldo:.2f}', saldo, extrato, numero_saques

def fazer_login():
    cpf = input("Digite seu CPF (só números): ")
    numero_conta = int(input("Digite o número da sua conta: "))
    
    usuario_encontrado = None
    for usuario in usuarios:
        if usuario['cpf'] == cpf:
            usuario_encontrado = usuario
            break
    
    if not usuario_encontrado:
        print(f"Usuário com CPF {cpf} não encontrado.")
        return None, None
    
    conta_encontrada = None
    for conta in contas_correntes:
        if conta['usuario']['cpf'] == cpf and conta['numero_conta'] == numero_conta:
            conta_encontrada = conta
            break
    
    if not conta_encontrada:
        print(f"A conta {numero_conta} não está vinculada ao usuário com CPF {cpf}.")
        return None, None
    
    print(f"Login ok para {usuario_encontrado['nome']} - Agência: {conta_encontrada['agencia']} - Conta: {conta_encontrada['numero_conta']}.")
    return usuario_encontrado, conta_encontrada

def escolher_login_ou_criar_conta():
    while True:
        opcao = input("Digite 'login' para entrar na sua conta ou 'criar' para nova conta: ").strip().lower()
        
        if opcao == 'login':
            return 'login'
        elif opcao == 'criar':
            return 'criar'
        else:
            print("Opção inválida. Por favor, digite 'login' ou 'criar'.")

opcao = escolher_login_ou_criar_conta()

if opcao == 'criar':
    nome = input("Digite seu nome: ")
    nascimento = input("Digite sua data de nascimento (DD/MM/AAAA): ")
    cpf = input("Digite seu CPF (só números): ")
    endereco = input("Digite seu endereço (logradouro, numero - bairro - cidade/estado): ")
    
    usuario = adicionar_usuario(nome, nascimento, cpf, endereco)
    if usuario:
        adicionar_conta(usuario)


    else:
        print("Não foi possível criar o usuário ou a conta.")

usuario_logado = None
if opcao == 'login':
    while not usuario_logado:
        usuario_logado, conta_logada = fazer_login()

saldo = 0
limite = 500
extrato = ""
numero_saques = 0
LIMITE_SAQUES = 3

while True:
    menu_valido = False
    while not menu_valido:
        menu = input('\nEscolha a opção: \n [d] - Depositar \n [s] - Sacar \n [e] - Extrato \n [q] - sair\n')
        
        if menu not in ['d', 's', 'e', 'q']:
            print('\nOpção inválida.')
        else:
            menu_valido = True

    if menu == 'd':
        valor = input('\nValor do depósito: ')
        try:
            valor = float(valor)
        except ValueError:
            print("\nDepósito só aceita valores numéricos.")
            continue

        mensagem, saldo, extrato = depositar_dinheiro(saldo, valor, extrato)
        print(mensagem)
        
    elif menu == 's':
        valor = input('\nValor do saque: ')
        try:
            valor = float(valor)
        except ValueError:
            print("\nSaque só aceita valores numéricos.")
            continue

        mensagem, saldo, extrato, numero_saques = sacar_dinheiro(valor=valor, limite=limite, saldo=saldo, extrato=extrato, limite_saques=LIMITE_SAQUES, numero_saques=numero_saques)
        print(mensagem)

    elif menu == 'e':
        mensagem, saldo = mostrar_extrato(saldo, extrato=extrato)
        print(mensagem)

    elif menu == 'q':
        break

print("\nOperações encerradas. Obrigado por usar nossos serviços!")

Digite 'login' para entrar na sua conta ou 'criar' para nova conta: criar
Digite seu nome: daniel
Digite sua data de nascimento (DD/MM/AAAA): 30112000
Digite seu CPF (só números): 1000
Digite seu endereço (logradouro, numero - bairro - cidade/estado): aa
Usuário daniel cadastrado com sucesso.
Conta criada para o usuário daniel - Agência: 0001 - Conta: 1.

Escolha a opção: 
 [d] - Depositar 
 [s] - Sacar 
 [e] - Extrato 
 [q] - sair
d

Valor do depósito: 100

Depósito ok. Seu saldo é: R$ 100.00

Escolha a opção: 
 [d] - Depositar 
 [s] - Sacar 
 [e] - Extrato 
 [q] - sair
e

================ EXTRATO ================
Depósito: R$ 100.00

Saldo: R$ 100.00

Escolha a opção: 
 [d] - Depositar 
 [s] - Sacar 
 [e] - Extrato 
 [q] - sair
s

Valor do saque: 10

Saque ok. Seu saldo é: R$ 90.00

Escolha a opção: 
 [d] - Depositar 
 [s] - Sacar 
 [e] - Extrato 
 [q] - sair
s

Valor do saque: 1000

Máximo para saque é 500.

Escolha a opção: 
 [d] - Depositar 
 [s] - Sacar 
 [e] - Extrato 
 [q] - sa